In [189]:
from sklearn.feature_extraction.text import TfidfVectorizer
import boto3
import numpy as np
import io
import sagemaker

s3 = boto3.resource('s3')
s3.meta.client.download_file('mltestdiego', 'SMSSpamCollection.csv', '/tmp/data.csv')

vectors = []
labels = []

with open('/tmp/data.csv') as f:
    content = f.readlines()
    counter = 0
    for line in content: 
        counter = counter + 1
        text = line.split(",")
        vectors.append(text[1])
        labels.append(text[0])
        # create the transform

#vectorizer_vectors = TfidfVectorizer()
#vectorizer_vectors.fit(vectors)

#with open('/tmp/data.csv') as f:
#    vector = vectorizer_vectors.transform(f)

labels = np.asarray(labels).astype('float32')



In [190]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
       results[i, sequence] = 1. 
    return results

    
#populate dictionary    
words = []
for sms in vectors:
    for word in text_to_word_sequence(sms):
        words.append(word)

data = [] 
# one hot encoding for each sentence
for sms in vectors:
    temp = one_hot(sms, len(words))
    data.append(temp)


#vectorize the all sms
train = vectorize_sequences(data, len(words))
print train.shape


print len(data)
print len(words)



(5555, 87413)
5555
87413


In [191]:
train_labels = labels[:4000] 
test_labels = labels[4000:]

print train.shape

train_data = train[:4000] 
test_data = train[4000:]

print train_data.shape
print test_data.shape

(5555, 87413)
(4000, 87413)
(1555, 87413)


In [192]:
from keras import models
from keras import layers
from keras.layers.core import Flatten

model = models.Sequential()
model.add(layers.Dense(16, activation='relu',input_shape=(87413,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy']
             )


In [193]:
history = model.fit(train_data,
                   labels[:4000],
                epochs=10,
                    batch_size=500,
                    validation_data=(test_data, labels[4000:])
                   )

Train on 4000 samples, validate on 1555 samples
Epoch 1/10
4000/4000 [==============================] - 6s 1ms/step - loss: 0.6584 - acc: 0.8560 - val_loss: 0.6143 - val_acc: 0.9704
Epoch 2/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5813 - acc: 0.9820 - val_loss: 0.5498 - val_acc: 0.9807
Epoch 3/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.5153 - acc: 0.9880 - val_loss: 0.4898 - val_acc: 0.9820
Epoch 4/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.4521 - acc: 0.9897 - val_loss: 0.4318 - val_acc: 0.9820
Epoch 5/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.3929 - acc: 0.9905 - val_loss: 0.3782 - val_acc: 0.9826
Epoch 6/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.3387 - acc: 0.9918 - val_loss: 0.3293 - val_acc: 0.9839
Epoch 7/10
4000/4000 [==============================] - 5s 1ms/step - loss: 0.2902 - acc: 0.9920 - val_loss: 0.2862 - val_acc: 0.9846
Epoch 8/10
400

In [196]:

sms = "SIX chances to win CASH! From 100 to 20000 pounds txt> CSH11 and send to 87575. Cost 150p/day 6days 16+ TsandCs apply Reply HL 4 info"
sms = "Call now and get 80% discount on your next pair of shoes! Call 8003344555 and get all CASH you need"

sentence = one_hot(sms, len(words))

sentences = [sentence]

vector = vectorize_sequences(sentences,len(words))


result = model.predict(vector)
print(result)

[[ 0.45844048]]


In [179]:
from keras.models import load_model

model.save_weights('/tmp/my_sms_model_weights.h5')

In [183]:
# serialize model to JSON
model_json = model.to_json()
with open("/tmp/sms_model.json", "w") as json_file:
    json_file.write(model_json)
    


In [184]:
%%bash    
aws s3 cp /tmp/sms_model.json s3://mltestdiego/sms/

upload: ../../../tmp/sms_model.json to s3://mltestdiego/sms/sms_model.json


In [180]:
%%bash

aws s3 cp /tmp/my_sms_model_weights.h5 s3://mltestdiego/sms/

upload: ../../../tmp/my_sms_model_weights.h5 to s3://mltestdiego/sms/my_sms_model_weights.h5


In [187]:
from keras.models import model_from_json
with open("/tmp/sms_model.json", "w") as json_file:
    model2test = model_from_json(model_json)


In [188]:
%%bash

python -c 'import keras; print(keras.__version__)'

2.1.2


Using TensorFlow backend.
